# Genetic Algorithm for Feature Selection:

Genetic algorithms are inspired by the process of natural selection, they are commonly used to generate high-quality solutions to optimization and search problems by relying on bio-inspired operators such as mutation, crossover and selection.

The example below presents a task where a Genetic Algorithm was used in order to come up with the best possible combination of explanatory varaibles useful for a predictive (classification) task.

In [41]:
import pandas as pd
import numpy as np

data = pd.read_csv('dataset.csv')

df = data.iloc[ 0: 432, : ] 
df = df.dropna(how='any')# train set
dfo = data.iloc[ 432: , : ]
dfo = dfo.dropna(how='any')# test set

#In the chuck below you will see how the algorithm optimizes for the GINI coefficient, and this is why the train and 
# test set have been split exactly in half as to have the same number of known and predicted results useful for a 
# valid FPR and TPR. You can though change the measure for which the algorithm optimizes. 

list_inputs = set(df.iloc[:,:-1].columns.values) # -1 because usually the target variable is the last column of the df. 
output_var = set(df.iloc[:,-1:].columns.values)  

In [49]:
#GENETIC ALGORITHM 

from deap import creator, base, tools, algorithms #GENETIC ALGORITHM LIBRARY - requirement: pip install deap
import random
from sklearn import metrics
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.metrics import classification_report, roc_curve, auc

X = df[list(set(list_inputs))].copy() # explanatory variables of your TRAINING set
y = df[list(set(output_var))].copy() # target variable of your TRAINING set

Xo = dfo[list(set(list_inputs))].copy() # explanatory variables of your TEST set
        
print ("GENETIC ALGORITHM FOR FEATURE SELECTION:")

#####
#SETING UP THE GENETIC ALGORITHM and CALCULATING STARTING POOL (STARTING CANDIDATE POPULATION)
#####
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)
toolbox = base.Toolbox()
toolbox.register("attr_bool", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=len(list_inputs))
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
def evalOneMax(individual):
    return sum(individual),

toolbox.register("evaluate", evalOneMax)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

NPOPSIZE = 65 #RANDOM STARTING POOL SIZE
population = toolbox.population(n=NPOPSIZE)



#####
#ASSESSING GINI ON THE STARTING POOL
#####
dic_gini={}
for i in range(np.shape(population)[0]):
    # TRASLATING DNA INTO LIST OF VARIABLES (1-81)
    var_model = []
    for j in range(np.shape(population)[0]):
        if (population[i])[j]==1:
            var_model.append(list(list_inputs)[j])
    # ASSESSING GINI INDEX FOR EACH INVIVIDUAL IN THE INITIAL POOL 
            
    X_train= X.copy()
    Y_train= y.copy()
    
    X_test = Xo.copy()

    ######
    # Insert here (lr = ...) the type of model you want to use. In this example I used an ADABoostClassifier & RF
    #####             
    lr = AdaBoostClassifier(RandomForestClassifier(n_estimators=1000,n_jobs=-1,max_depth=None,max_features="sqrt",class_weight="balanced",
                                     criterion="gini",oob_score=True,random_state=None))
    model=lr.fit(X_train,Y_train.values.ravel())   
    Y_predict=model.predict_proba(X_test)
    Y_predict=Y_predict[:,1]
    ######
            
    ######
    # CHANGE_HERE - START: GINI
    #####                
    fpr, tpr, thresholds = metrics.roc_curve(Y_train, Y_predict)
    auc = metrics.auc(fpr, tpr)
    gini_power = abs(2*auc-1)
        
    ######
    # CHANGE_HERE - END
    #####                
    
    gini=str(gini_power)+";"+str(population[j]).replace('[','').replace(', ','').replace(']','')
    dic_gini[gini]=population[j]   
list_gini=sorted(dic_gini.keys(),reverse=True)

#GENETIC ALGORITHM MAIN LOOP - START
# - ITERATING MANY TIMES UNTIL NO IMPROVMENT HAPPENS IN ORDER TO FIND THE OPTIMAL SET OF CHARACTERISTICS (VARIABLES)

sum_current_gini=0.0
sum_current_gini_1=0.0
sum_current_gini_2=0.0
first=0    
OK = 1
a=0
while OK:  #REPEAT UNTIL IT DO NOT IMPROVE, AT LEAST A LITLE, THE GINI IN 2 GENERATIONS
    a=a+1
    print ('loop ', a)
    OK=0

    ####
    # GENERATING OFFSPRING - START
    ####
    offspring = algorithms.varAnd(population, toolbox, cxpb=0.5, mutpb=0.1) #CROSS-X PROBABILITY = 50%, MUTATION PROBABILITY=10%
    fits = toolbox.map(toolbox.evaluate, offspring)
    for fit, ind in zip(fits, offspring):
        ind.fitness.values = fit
    population =toolbox.select(offspring, k=len(population))
    ####
    # GENERATING OFFSPRING - END
    ####

    sum_current_gini_2=sum_current_gini_1
    sum_current_gini_1=sum_current_gini
    sum_current_gini=0.0

    #####
    #ASSESSING GINI ON THE OFFSPRING - START
    #####
    for j in range(np.shape(population)[0]): 
        if population[j] not in dic_gini.values(): 
            var_model = [] 
            for i in range(np.shape(population)[0]): 
                if (population[j])[i]==1:
                    var_model.append(list(list_inputs)[i])
            
            X_train=df[var_model]
            Y_train=df[list(output_var)] # or whatever you have called it in your own data set. 
            
            X_test = dfo[var_model]
            
            ######
            # Same model as before (lr = ...) used to predict probabilities of TEST set. 
            #####            
            lr = AdaBoostClassifier(RandomForestClassifier(n_estimators=1000,n_jobs=-1,max_depth=None,max_features="sqrt",class_weight="balanced",
                                     criterion="gini",oob_score=True,random_state=None))
            model=lr.fit(X_train,Y_train.values.ravel())   
            Y_predict=model.predict_proba(X_test)
            Y_predict=Y_predict[:,1]
            ######

            # CHANGE_HERE - START: GINI
            #####                       
            fpr, tpr, thresholds = metrics.roc_curve(Y_train, Y_predict)
            auc = metrics.auc(fpr, tpr)
            gini_power = abs(2*auc-1)
            
            # CHANGE_HERE - END
            #####                       
           
            gini=str(gini_power)+";"+str(population[j]).replace('[','').replace(', ','').replace(']','')
            dic_gini[gini]=population[j]  
    #####
    #ASSESSING GINI ON THE OFFSPRING - END
    #####

    #####
    #SELECTING THE BEST FITTED AMONG ALL EVER CREATED POPULATION AND CURRENT OFFSPRING - START
    #####           
    list_gini=sorted(dic_gini.keys(),reverse=True)
    population=[]
    for i in list_gini[:NPOPSIZE]:
        population.append(dic_gini[i])
        gini=float(i.split(';')[0])
        sum_current_gini+=gini
    #####
    #SELECTING THE BEST FITTED AMONG ALL EVER CREATED POPULATION AND CURRENT OFFSPRING - END
    #####           
      
    #HAS IT IMPROVED AT LEAST A LITLE THE GINI IN THE LAST 2 GENERATIONS
    print('sum_current_gini=', sum_current_gini, 'sum_current_gini_1=', sum_current_gini_1, 'sum_current_gini_2=', sum_current_gini_2)
    if(sum_current_gini>sum_current_gini_1+0.0001 or sum_current_gini>sum_current_gini_2+0.0001):
        OK=1
#####
#GENETIC ALGORITHM MAIN LOOP - END
#####


gini_max=list_gini[0]        
gini=float(gini_max.split(';')[0])
features=gini_max.split(';')[1]


####
# PRINTING OUT THE LIST OF FEATURES
#####
f=0
for i in range(len(features)):
    if features[i]=='1':
        f+=1
        print ('feature ', f, ':', list(list_inputs)[i]) #this is the BEST combination of explanatory varaibles
print ('gini: ', gini)

GENETIC ALGORITHM FOR FEATURE SELECTION:
loop  1
sum_current_gini= 5.1733126083575 sum_current_gini_1= 0.0 sum_current_gini_2= 0.0
loop  2
sum_current_gini= 6.0194894481976995 sum_current_gini_1= 5.1733126083575 sum_current_gini_2= 0.0
loop  3
sum_current_gini= 6.456387875889298 sum_current_gini_1= 6.0194894481976995 sum_current_gini_2= 5.1733126083575
loop  4
sum_current_gini= 7.5939499013570995 sum_current_gini_1= 6.456387875889298 sum_current_gini_2= 6.0194894481976995
loop  5
sum_current_gini= 7.8506606085983 sum_current_gini_1= 7.5939499013570995 sum_current_gini_2= 6.456387875889298
loop  6
sum_current_gini= 8.676451246489602 sum_current_gini_1= 7.8506606085983 sum_current_gini_2= 7.5939499013570995
loop  7
sum_current_gini= 9.454594368387005 sum_current_gini_1= 8.676451246489602 sum_current_gini_2= 7.8506606085983
loop  8
sum_current_gini= 10.311592036828003 sum_current_gini_1= 9.454594368387005 sum_current_gini_2= 8.676451246489602
loop  9
sum_current_gini= 11.034495127639001 s